In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
d = pd.read_csv('../data/adult2_train.csv')

In [3]:
d.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,6,77516,11,13,4,0,1,4,1,2174,0,40,38,0
1,50,5,83311,11,13,2,3,0,4,1,0,0,13,38,0
2,38,3,215646,8,9,0,5,1,4,1,0,0,40,38,0
3,53,3,234721,6,7,2,5,0,2,1,0,0,40,38,0
4,28,3,338409,11,13,2,9,5,2,0,0,0,40,4,0


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.ensemble import VotingClassifier

In [5]:
X_train, X_test, y_train, y_test = train_test_split(d.drop('14', axis=1), d['14'], test_size=0.33, 
                                                           random_state=42, stratify=d['14'])

In [6]:
X_train.shape, X_test.shape

((21815, 14), (10746, 14))

In [7]:
import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [8]:
clfXGB = xgb.XGBClassifier(n_estimators=380, nthread=-1)
# clfXGB.fit(X_train, y_train)

In [9]:
clfLGB = LGBMClassifier(n_estimators=380, nthread=-1)
clfLGB2 = LGBMClassifier(n_estimators=230, nthread=-1)
# clfLGB.fit(X_train, y_train)

In [10]:
clfCT = CatBoostClassifier(thread_count=4, num_trees=380)
clfCT2 = CatBoostClassifier(thread_count=4, num_trees=200)
# clfCT.fit(X_train, y_train)

In [11]:
clfMain = VotingClassifier(estimators=[('xgb', clfXGB), ('lgb', clfLGB), ('lgb2', clfLGB2), 
                                       ('clfCT', clfCT), ('clfCT2', clfCT2)], voting='soft')

In [12]:
clfMain.fit(X_train, y_train)

VotingClassifier(estimators=[('xgb', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=380,
       n_jobs=1, nthread=-1, objective='binary:logistic', random_s...ifier object at 0x10fcb5f98>), ('clfCT2', <catboost.core.CatBoostClassifier object at 0x10fcb5f28>)],
         n_jobs=1, voting='soft', weights=None)

In [13]:
accuracy_score(clfMain.predict(X_train), y_train)

0.89901443960577587

In [14]:
accuracy_score(clfMain.predict(X_test), y_test)

0.8761399590545319

In [15]:
f1_score(clfMain.predict(X_train), y_train)

0.77144932046892833

In [16]:
f1_score(clfMain.predict(X_test), y_test)

0.72020180786209798

In [19]:
print(classification_report(clfMain.predict(X_train), y_train))

             precision    recall  f1-score   support

          0       0.96      0.91      0.94     17429
          1       0.71      0.85      0.77      4386

avg / total       0.91      0.90      0.90     21815



In [20]:
print(classification_report(clfMain.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.94      0.90      0.92      8577
          1       0.66      0.79      0.72      2169

avg / total       0.89      0.88      0.88     10746

